# Parameters

In [39]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 64
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
train_datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


# Set up TrainSoundsDataset

# Set up PsychophysicsSoundsDataset

In [8]:
f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')

In [9]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [10]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [26]:
def getPsychophysics2017WCleanCochleagrams():
    
    cochleagrams_clean = []
   
    cochleagrams = []
    for batch_ii in range(0,15300,100):
        hdf5_path = '/mnt/smb/locker/issa-locker/users/Erica/cgrams_for_noise_robustness_analysis/PsychophysicsWord2017W_clean/batch_'+str(batch_ii)+'_to_'+str(batch_ii+100)+'.hdf5'
        with h5py.File(hdf5_path, 'r') as f_in:
            cochleagrams += list(f_in['data'])

    return cochleagrams
clean_in = getPsychophysics2017WCleanCochleagrams()
clean_in = np.array(clean_in)

In [34]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

## Save hdf5 of psychophysics word clean -- rather than individual batch hdf5s

In [39]:

import numpy as np
import h5py


out_base_path = '/mnt/smb/locker/issa-locker/users/Erica/'

out_file_name = 'PsychophysicsWord2017W_clean.hdf5'
num_cgrams = len(clean_in)

with h5py.File(out_base_path+out_file_name, 'x') as f_out:
        label_data_set = f_out.create_dataset("labels",(num_cgrams,), dtype="int32")
        cgram_dataset = f_out.create_dataset("data", (num_cgrams, 164*400), dtype='float32')
       

        label_data_set = labels

        cgram_dataset= clean_in * 1000

        

## Save hdf5 of psychophysics word clean just WSJ -- rather than individual batch hdf5s

In [11]:
orig_dset = []
for _orig_dset in f_metadata['orig_dset']:
    _orig_dset = str(_orig_dset, 'utf-8')
    _orig_dset = 'WSJ' if 'WSJ' in _orig_dset else 'Timit'
    orig_dset.append(_orig_dset)
orig_dset = np.array(orig_dset)

In [12]:
wsj_inds = np.where(orig_dset == 'WSJ')

In [16]:
import numpy as np
import h5py


out_base_path = '/mnt/smb/locker/issa-locker/users/Erica/'

out_file_name = 'PsychophysicsWord2017W_clean_wsjOnly.hdf5'
num_cgrams = len(wsj_inds[0])

with h5py.File(out_base_path+out_file_name, 'x') as f_out:
        label_data_set = f_out.create_dataset("labels",(num_cgrams,), dtype="int32")
        cgram_dataset = f_out.create_dataset("data", (num_cgrams, 164*400), dtype='float32')
       
        with h5py.File('/mnt/smb/locker/issa-locker/users/Erica/PsychophysicsWord2017W_clean.hdf5','r') as f_in:
            
            
            
            label_data_set = f_in['labels'][list(wsj_inds[0])]

            cgram_dataset= f_in['data'][list(wsj_inds[0])]



## Make combined hdf5 training dataset in random order

In [18]:
num_cgrams = 0
for data_file in ['/mnt/smb/locker/issa-locker/users/Erica/PsychophysicsWord2017W_clean_wsjOnly.hdf5', '/mnt/smb/locker/issa-locker/users/Erica/seed_542_word_clean_random_order.hdf5']:
    with h5py.File(data_file, 'r') as f_in:
        num_cgrams += len(f_in['data'])

In [20]:
print(num_cgrams)

55188


In [25]:


available_indices = list(range(num_cgrams))

indice_order = {} #this should be a dictionary that tells for a given batch file and index in that batch which indice in the large hdf5 this cgram is mapped onto
out_base_path = '/mnt/smb/locker/issa-locker/users/Erica/'

out_file_name = 'training_dataset_random_order.hdf5'

with h5py.File(out_base_path+out_file_name, 'x') as f_out:
        label_data_set = f_out.create_dataset("labels",(num_cgrams,), dtype="int32")
        cgram_dataset = f_out.create_dataset("data", (num_cgrams, 164*400), dtype='float32')
        

        for data_file in ['/mnt/smb/locker/issa-locker/users/Erica/PsychophysicsWord2017W_clean_wsjOnly.hdf5', '/mnt/smb/locker/issa-locker/users/Erica/seed_542_word_clean_random_order.hdf5']:
            with h5py.File(data_file, 'r') as f_in:

                for ii in range(len(f_in['data'])):
                    if ii %100 == 0:
                        print(ii)

                    #Choose indice in large hdf5 for this cgram
                    index_into_available_indices= np.random.choice(range(len(available_indices)))

                    #not the index into the large hdf5 -> this is the index into the available_indices array
                    chosen_indice = available_indices[index_into_available_indices]
                    dict_key = data_file+'_'+ str(ii)
                    indice_order[dict_key] = chosen_indice #add to dictionary

                    available_indices.pop(index_into_available_indices) #remove chosen indice from available indices
                    if data_file == '/mnt/smb/locker/issa-locker/users/Erica/seed_542_word_clean_random_order.hdf5':
                        label_data_set[chosen_indice] = f_in['labels'][ii]*1000
                    else:
                        label_data_set[chosen_indice] = f_in['labels'][ii]
                    
                    
                    cgram_dataset[chosen_indice] = f_in['data'][ii]


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
48

In [ ]:
# Need to save indice order dict

In [28]:
np.save(out_base_path + 'indice_order_dict', indice_order)